In [1]:
# Install liblinear: sudo apt-get install python-liblinear
# from liblinearutil import *
import numpy as np

In [2]:
def read_problem(file_name):
    y = []
    x = []
    feature_len = 22
    for line in open(file_name):
        tmp = line.split(' ',1)
        y = y + [int(tmp[0])]
        vec = np.zeros(feature_len)
        for each in tmp[1].split():
            index,val = each.split(':')
            vec[int(index)-1] = float(val)
        x.append(vec)
    y = np.array(y)
    x = np.array(x)
    return y,x

# Gradient ascent step for dual variables

In [22]:
def get_tp_tn(x,y,w):
    h = x.dot(w)
    ind = np.where(h>0)
    y1 = -1*np.ones_like(y)
    y1[ind] = 1
    tp = sum(np.multiply(1+y,1+y1)/4)
    tn = sum(np.multiply(1-y,1-y1)/4)
    return tp,tn

def get_projection(alpha,beta):
    if alpha>0 and beta>0 and alpha*beta>=1.0/4:
        return alpha,beta
    coef = [16,-16*alpha,0,4*beta,-1]
    roots = np.roots(coef)
    im = np.imag(roots)
    re = np.real(roots)
    for r,i in zip(re,im):
        if i==0 and r>0:
            r1 = 1.0/(4*r)
            return np.round(r,decimals=2),np.round(r1,decimals=2)

# Change it to gradient descent
def gradient_ascent(x,y,w,a,b,alpha=0.1,maxiter=50,tolerance=0.0001):
    n = x.shape[0]
    n_pos = len([i for i in range(n) if y[i]==1])
    n_neg = n-n_pos
    ind_pos = [i for i in range(n) if y[i]>0]
    ind_neg = [i for i in range(n) if y[i]<0]
    x_pos = x[ind_pos]
    x_neg = x[ind_neg]
    tpr,tnr = get_tp_tn(x,y,w)
    tpr /= float(n_pos)
    tnr /= float(n_neg)
    print('-------------Starting Gradient Ascent: step={},tolerance={}------------'.format(alpha,tolerance))
    print('n_pos={},n_neg={},TPR ={},TNR={}'.format(n_pos,n_neg,tpr,tnr))
    for it in range(maxiter):
#         alpha = 1.0/(it+1)
        # for positives
        tmp = 1 - (2*n_pos)/(a*n)*x_pos.dot(w)
        j = np.where(tmp>0)[0]
        tmp = np.zeros_like(tmp)
        tmp[j]=1.0
        pos_violation = sum(tmp)
#         print('pos-violation={}'.format(sum(tmp)))
        ga = sum(tmp)/n_pos-1
        # for negatives
        tmp = 1 + (2*n_neg)/(b*n)*x_neg.dot(w)
        j = np.where(tmp>0)[0]
        tmp = np.zeros_like(tmp)
        tmp[j]=1.0
        neg_violation = sum(tmp)
#         print('neg-violation={}'.format(sum(tmp)))
        gb = sum(tmp)/n_neg-1
        print('Iteration: {},pos violation = {},neg violation = {}, grad_pos = {},grad_neg = {}'.format(it,pos_violation,neg_violation,ga,gb))
        # Update a,b
        if np.sqrt(ga*ga+gb*gb)<tolerance:
            print('------------Returning(small gradient): grad_pos = {} grad_neg = {}-----------'.format(ga,gb))
            return a,b
        a += alpha*ga
        b += alpha*gb
        # Projection to a,b \in R+ \intersect ab>1/4
        a,b = get_projection(a,b)
#         print(a,b)
    print('---------------Returning(normal): grad_pos = {} grad_neg = {}-------------'.format(ga,gb))
    return a,b

## Liblinear for primal variable w

In [4]:
# customized liblinear
import sys
sys.path.insert(0, "/home/debojyoti/opt/liblinear-2.1")
from ppython import liblinear
from ppython.liblinear import *
from ppython.liblinearutil import *

In [23]:
def modifyx(x,y,a,b,n,n_pos,n_neg):
    scale_pos = float(2*n_pos)/(a*n)
    scale_neg = float(2*n_neg)/(b*n)
    for i in range(len(y)):
        if y[i]==1:
            x[i].update((key,val*scale_pos) for key,val in x[i].items())
        else:
            x[i].update((key,val*scale_neg) for key,val in x[i].items())
    return x
# init section
a,b = 0.5,0.5
y_lst,x_lst = read_problem('./data/ijcnn1.tr') # x as numpy list
n_dim = len(x_lst[0])
w = np.random.uniform(low=-1,high=1,size=(n_dim,))
# w = np.zeros(n_dim)
y_orig,x_orig = svm_read_problem('./data/ijcnn1.tr') # x in liblinear compatible dictionary format
n = len(y_lst)
n_pos = len([i for i in range(n) if y_lst[i]==1])
n_neg = n-n_pos
c = 5.0
# Iterative section: Gradient Ascent & Liblinear
for i in range(10):
    a,b = gradient_ascent(x_lst,y_lst,w,a,b) #Gradient ascent
    c1 = float(a)/n_pos
    c2 = float(b)/n_neg
    param = parameter('-s 3 -w1 {} -w-1 {} -c {}'.format(c1,c2,c))
    x = modifyx(x_orig,y_orig,a,b,n,n_pos,n_neg)
    prob = problem(y_orig,x)
    print('--------------------Running SVM, a = {}, b = {}----------------------'.format(a,b))
    model = train(prob,param)
    # Get model parameters
    w = model.get_decfun()[0]

-------------Starting Gradient Ascent: step=0.1,tolerance=0.0001------------
n_pos=3415,n_neg=31585,TPR =0.585358711567,TNR=0.461928130442
Iteration: 0,pos violation = 3415.0,neg violation = 22185.0, grad_pos = 0.0,grad_neg = -0.297609624822
Iteration: 1,pos violation = 3415.0,neg violation = 22124.0, grad_pos = 0.0,grad_neg = -0.299540921323
Iteration: 2,pos violation = 3415.0,neg violation = 22060.0, grad_pos = 0.0,grad_neg = -0.30156719962
Iteration: 3,pos violation = 3415.0,neg violation = 22007.0, grad_pos = 0.0,grad_neg = -0.303245211334
Iteration: 4,pos violation = 3415.0,neg violation = 21884.0, grad_pos = 0.0,grad_neg = -0.307139464936
Iteration: 5,pos violation = 3415.0,neg violation = 21826.0, grad_pos = 0.0,grad_neg = -0.308975779642
Iteration: 6,pos violation = 3415.0,neg violation = 21753.0, grad_pos = 0.0,grad_neg = -0.311287003324
Iteration: 7,pos violation = 3415.0,neg violation = 21693.0, grad_pos = 0.0,grad_neg = -0.313186639227
Iteration: 8,pos violation = 3415.0,ne